In [1]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from PIL import Image
from scipy import sparse
import cv2
from pymatreader import read_mat
# from extract_graph import dic_to_sparse
from util import get_path
from plotutil import show_im,overlap, show_im_rgb, plot_nodes, plot_nodes_from_list,plot_t_tp1
from extract_graph import generate_graph_tab_from_skeleton,generate_nx_graph_from_skeleton,generate_skeleton,clean
import networkx as nx
from node_id import second_identification, whole_movement_identification,first_identification,relabel_nodes, clean_nodes, orient
plate=13
date1='0703_1157'
date2='0703_1557'
date3='0703_1957'
row=6
column=10


In [2]:
imtab1=np.load(f'Data/imbackrem_{date1}_{plate}_{row}_{column}.npy')
imtab2=np.load(f'Data/imbackrem_{date2}_{plate}_{row}_{column}.npy')
imtab3=np.load(f'Data/imbackrem_{date3}_{plate}_{row}_{column}.npy')
skeleton1=clean(np.load(f'Data/skeletonized_{date1}_{plate}_{row}_{column}.npy'))
skeleton2=clean(np.load(f'Data/skeletonized_{date2}_{plate}_{row}_{column}.npy'))
skeleton3=clean(np.load(f'Data/skeletonized_{date3}_{plate}_{row}_{column}.npy'))
width2=np.load(f'Data/width_{date2}_{plate}_{row}_{column}.npy')

In [3]:
agraph_tab1=generate_graph_tab_from_skeleton(skeleton1)
agraph_tab2=generate_graph_tab_from_skeleton(skeleton2)
agraph_tab3=generate_graph_tab_from_skeleton(skeleton3)
anx_graph_t0m1,apos_t0m1=generate_nx_graph_from_skeleton(skeleton1)
anx_graph_t0,apos_t0=generate_nx_graph_from_skeleton(skeleton2)
anx_graph_t0p1,apos_t0p1=generate_nx_graph_from_skeleton(skeleton3)

In [4]:

# pos_t0p1_clean,nx_graph_t0p1_clean,nx_graph_t0_clean,corresp_tips=second_identification(anx_graph_t0,anx_graph_t0p1,apos_t0,apos_t0p1,50)
# pos_t0_clean,nx_graph_t0_clean_clean,nx_graph_t0m1_clean,corresp_tips=second_identification(anx_graph_t0m1,nx_graph_t0_clean,apos_t0m1,apos_t0,50)
# skeleton_clean_t0m1=generate_skeleton(nx_graph_t0m1_clean).todense()
# skeleton_clean_t0=generate_skeleton(nx_graph_t0_clean_clean).todense()
# skeleton_clean_t0p1=generate_skeleton(nx_graph_t0p1_clean).todense()

In [23]:
def second_identification(nx_graph_tm1,nx_graph_t,pos_tm1,pos_t,length_id=50):
    corresp,to_remove=first_identification(nx_graph_tm1,nx_graph_t,pos_tm1,pos_t)
    nx_graph_tm1=clean_nodes(nx_graph_tm1,to_remove,pos_tm1)
    pos_t,nx_graph_t=relabel_nodes(corresp,nx_graph_t,pos_t)
    corresp_tips={node:node for node in corresp.keys()}
    tips = [node for node in nx_graph_tm1.nodes if nx_graph_tm1.degree(node)==1]
    for tip in tips:
#         print('tip',pos_tm1[tip],tip)
        mini=np.inf
        for edge in nx_graph_t.edges:
            pixel_list=nx_graph_t.get_edge_data(*edge)['pixel_list']
            distance=np.min(np.linalg.norm(np.array(pixel_list)-np.array(pos_tm1[tip]),axis=1))
            if distance<mini:
                mini=distance
                right_edge = edge
        origin = np.array(orient(nx_graph_tm1.get_edge_data(*list(nx_graph_tm1.edges(tip))[0])['pixel_list'],pos_tm1[tip]))
        origin_vector = origin[0]-origin[-1]
        branch=np.array(orient(nx_graph_t.get_edge_data(*right_edge)['pixel_list'],pos_t[right_edge[0]]))
        candidate_vector = branch[-1]-branch[0]
        dot_product = np.dot(origin_vector,candidate_vector)
        if dot_product>=0:
            root=right_edge[0]
            next_node=right_edge[1]
        else:
            root=right_edge[1]
            next_node=right_edge[0]
        last_node=root
        current_node=next_node
        last_branch=np.array(orient(nx_graph_t.get_edge_data(root,next_node)['pixel_list'],pos_t[current_node]))
        while nx_graph_t.degree(current_node)!=1: #Careful : if there is a cycle with low angle this might loop indefinitely but unprobable
            mini=np.inf
            origin_vector = last_branch[0]-last_branch[min(length_id,len(last_branch)-1)]
            unit_vector_origin = origin_vector / np.linalg.norm(origin_vector)
            candidate_vectors=[]
            for neighbours_t in nx_graph_t.neighbors(current_node):
                if neighbours_t!=last_node:
                    branch_candidate=np.array(orient(nx_graph_t.get_edge_data(current_node,neighbours_t)['pixel_list'],pos_t[current_node]))
                    candidate_vector = branch_candidate[min(length_id,len(branch_candidate)-1)]-branch_candidate[0]
                    unit_vector_candidate = candidate_vector / np.linalg.norm(candidate_vector)
                    candidate_vectors.append(unit_vector_candidate)
                    dot_product = np.dot(unit_vector_origin, unit_vector_candidate)
                    angle = np.arccos(dot_product)
                    if angle<mini:
                        mini=angle
                        next_node=neighbours_t
#                     print('angle',dot_product,pos_t[last_node],pos_t[current_node],pos_t[neighbours_t],angle/(2*np.pi)*360)
#!!!bug may happen here if two nodes are direct neighbours : I would nee to check further why it the case, optimal segmentation should avoid this issue.
# This is especially a problem for degree 4 nodes. Maybe fuse nodes that are closer than 3 pixels.
            if len(candidate_vectors)<2:
                print(nx_graph_t.degree(current_node),pos_t[current_node],[node for node in nx_graph_t.nodes if nx_graph_t.degree(node)==2])
            competitor = np.arccos(np.dot(candidate_vectors[0],-candidate_vectors[1]))
#             print('competitor',competitor/(2*np.pi)*360)
            if mini<competitor:
                current_node,last_node=next_node,current_node
            else:
                corresp_tips[tip]=current_node
                break
        corresp_tips[tip]=current_node
    pos_t,nx_graph_t=relabel_nodes(corresp_tips,nx_graph_t,pos_t)
    return(pos_t,nx_graph_t,nx_graph_tm1,corresp_tips)
                

In [26]:
apos_t0,anx_graph_t0,anx_graph_t0m1,corresp_tips=second_identification(anx_graph_t0m1,anx_graph_t0,apos_t0m1,apos_t0,50)
apos_t0p1,anx_graph_t0p1,anx_graph_t0,corresp_tips=second_identification(anx_graph_t0,anx_graph_t0p1,apos_t0,apos_t0p1,50)



skeleton_clean_t0m1=generate_skeleton(anx_graph_t0m1).todense()
skeleton_clean_t0=generate_skeleton(anx_graph_t0).todense()
skeleton_clean_t0p1=generate_skeleton(anx_graph_t0p1).todense()

set()
set()


In [34]:
tip_corresp=whole_movement_identification(anx_graph_t0m1,anx_graph_t0,apos_t0m1,apos_t0,50)

In [39]:
# list_anastomosis = pinpoint_anastomosis(anx_graph_t0m1,anx_graph_t0)
list_tips = [node for node in anx_graph_t0m1.nodes if anx_graph_t0m1.degree(node)==1]
list_tips2 = [node for node in anx_graph_t0.nodes if anx_graph_t0.degree(node)==1]

In [42]:
list_nodes_t

In [44]:
def relabel(tip):
    return((list_tips+list_tips2).index(tip))

In [71]:
test=(dilated2>60)*60+((dilated2<=60)*dilated2*(dilated2>=20))+(dilated2<20)*20

In [101]:
plot_t_tp1(list_tipstm1,list_tipst+list_tipstm1,apos_t0m1,apos_t0,dilated2,imtab2,relabel_t=relabel,relabel_tp1=lambda x : relabel(tip_corresp[x]) if x in tip_corresp.keys() else relabel(x))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [102]:
plot_t_tp1([],[],apos_t0m1,apos_t0,dilated2,imtab2,relabel_t=relabel,relabel_tp1=lambda x : relabel(tip_corresp[x]) if x in tip_corresp.keys() else relabel(x))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [73]:
plot_t_tp1(list_tips,list_tips+list_tips2,apos_t0m1,apos_t0,dilated2,imtab2,relabel_t=relabel,relabel_tp1=lambda x : relabel(tip_corresp[x]) if x in tip_corresp.keys() else relabel(x))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [63]:
list_tipstm1=[tip for tip in list_tips if (apos_t0m1[tip][0]<=2500 and apos_t0m1[tip][0]>=500 and apos_t0m1[tip][1]<=1500)]

In [65]:
list_tipst=[tip for tip in list_tips2 if (apos_t0[tip][0]<=2500 and apos_t0[tip][0]>=500 and apos_t0[tip][1]<=1500)]

In [100]:
def plot_t_tp1(node_list_t,node_list_tp1,pos_t,pos_tp1,imt,imtp1,relabel_t=lambda x:x,relabel_tp1=lambda x:x):
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.imshow(imtp1, cmap='gray',interpolation='none',extent=[0,imtp1.shape[1]*pixel_conversion_factor,imtp1.shape[0]*pixel_conversion_factor,0])
    width=ax.imshow(imt, cmap='jet', alpha=0.5,interpolation='none',extent=[0,imtp1.shape[1]*pixel_conversion_factor,imtp1.shape[0]*pixel_conversion_factor,0])
    bbox_props1 = dict(boxstyle="circle", fc="grey")
    bbox_props2 = dict(boxstyle="circle", fc="white")
    for node in node_list_t:
        t = ax.text(pos_t[node][1]*pixel_conversion_factor, pos_t[node][0]*pixel_conversion_factor, str(relabel_t(node)), ha="center", va="center",
                    size=5,
                    bbox=bbox_props1)
    for node in node_list_tp1:
        if node in pos_tp1.keys():
            t = ax.text(pos_tp1[node][1]*pixel_conversion_factor, pos_tp1[node][0]*pixel_conversion_factor, 
                        str(relabel_tp1(node)), ha="center", va="center",
                        size=5,
                        bbox=bbox_props2)
    ax.set_xlabel('x position (μm)')
    ax.set_ylabel('y position (μm)')
    cbar = fig.colorbar(width,ax=ax)
    cbar.set_label('Hyphal width (μm)')
    plt.show()

In [6]:
def pinpoint_anastomosis(nx_graph_tm1,nx_grapht):
    anastomosis=[]
    tips = [node for node in nx_graph_tm1.nodes if nx_graph_tm1.degree(node)==1]
    for node in tips:
        if nx_grapht.degree(node)>=3:
            anastomosis.append(node)
    return(anastomosis)

In [59]:
kernel = np.ones((5,5),np.uint8)
pixel_conversion_factor = 1.725
dilated2 = cv2.dilate(width2.astype(np.uint8) *pixel_conversion_factor,kernel,iterations = 1)

In [ ]:
plot_t_tp1(list_tips,list_tips,apos_t0m1,apos_t0,dilated2,imtab2)

In [10]:
plot_t_tp1(anx_graph_t0m1.nodes,anx_graph_t0.nodes,apos_t0m1,apos_t0,skeleton1,imtab1)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [11]:
distance = [np.linalg.norm(np.array(apos_t0m1[tip])-np.array(apos_t0[tip]))*1.725/(4*3600) for tip in list_tips]

In [12]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax.hist(distance)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
plot_t_tp1(list_tips2,list_tips2,apos_t0,apos_t0p1,imtab2,imtab3)

In [3]:
date1='0709_1557'
date2='0705_1557'
date3='0705_1957'


In [4]:
get_path(date1,plate,True)

'//sun.amolf.nl/shimizu-data/home-folder/oyartegalvez/Drive_AMFtopology/PRINCE/20200709_1557_Plate13/Analysis/Skeleton.mat'

In [5]:
mat_skel=read_mat(get_path(date1,plate,True))['skel']

In [6]:
from extract_graph import dic_to_sparse, from_sparse_to_graph, generate_nx_graph, prune_graph


In [7]:
dic_skel=dic_to_sparse(mat_skel)

In [10]:
skeleton_doc1=sparse.dok_matrix(dic_skel)

In [9]:
skeleton_doc.shape

NameError: name 'skeleton_doc' is not defined

In [73]:
skeleton_doc_small[500]

IndexError: row index (30) out of range

In [59]:
skeleton_doc_small=skeleton_doc1[15000:20000,15000:20000]

# graph_tab_small=from_sparse_to_graph(skeleton_doc_small)
# nx_graph_small,pos_small=generate_nx_graph(graph_tab_small)

In [63]:
def dilate(sparse_matrix_doc):
    dilated = sparse.dok_matrix(sparse_matrix_doc.shape, dtype=bool)
    for pos in sparse_matrix_doc.keys():
        positions=[pos,(pos[0],pos[1]+1),(pos[0],pos[1]-1),(pos[0]-1,pos[1]-1),(pos[0]+1,pos[1]-1),(pos[0]+1,pos[1]+1),(pos[0]-1,pos[1]+1),(pos[0]-1,pos[1]),(pos[0]+1,pos[1])]
        for position in positions:
            if position[0]<sparse_matrix_doc.shape[0] and position[1]<sparse_matrix_doc.shape[1] and position[0]>=0 and position[0]>=0:
                dilated[position]=1
    return(dilated)


def zeroToOne(thin_image,i,j):
    p2 = thin_image[i-1,j-1]
    p3 = thin_image[i-1,j]
    p4 = thin_image[i-1,j+1]
    p5 = thin_image[i,j+1]
    p6 = thin_image[i+1,j+1]
    p7 = thin_image[i+1,j]
    p8 = thin_image[i+1,j-1]
    p9 = thin_image[i,j-1]
    count = 0;
    endpoint = 0;
    if(p2+p3+p4+p5+p6+p7+p8+p9 == 1):
        endpoint = 1
    if(p2==0 and p3==1):
        count = count + 1
    if(p3==0 and p4==1):
        count = count + 1
    if(p4==0 and p5==1):
        count = count + 1
    if(p5==0 and p6==1):
        count = count + 1
    if(p6==0 and p7==1):
        count = count + 1
    if(p7==0 and p8==1):
        count = count + 1
    if(p8==0 and p9==1):
        count = count + 1
    if(p9==0 and p2==1):
        count = count + 1
    return count,endpoint




def stentiford(image):
    # Make copy of the image so that original image is not lost
    thin_image=sparse.dok_matrix((image.shape[0]+2,image.shape[1]+2), dtype=bool)
    thin_image[1:image.shape[0]+1,1:image.shape[1]+1]=image.copy()
    check = 2
    template = 1
    outImage = 1
    # Perform iterations as long as there are pixels marked for deletion
    iteration = 0
    total_changes = 0
    while(outImage):
        # Make outImage empty
        outImage = []
        changes = 0
        iteration = iteration + 1 
        # Loop through the pixels of the thin_image
        for pos in thin_image.keys():
            i=pos[0]
            j=pos[1]
            p0 = thin_image[i,j]
            p1 = thin_image[i-1,j]
            p2 = thin_image[i,j+1]
            p3 = thin_image[i+1,j]
            p4 = thin_image[i,j-1]
            if(template==1):
                template_match = (p1==0 and p3==1)
            if(template==2):
                template_match = (p2==1 and p4==0)
            if(template==3):
                template_match = (p1==1 and p3==0)
            if(template==4):
                template_match = (p2==0 and p4==1)
            connectivity, isEndpoint = zeroToOne(thin_image,i,j)
            if(template_match==1):
                if(connectivity == 1):
                    if(isEndpoint== 0):
                        outImage.append((i,j))


        # Delete the pixels marked for deletion
        for i,j in outImage:
            thin_image[i,j] = 0
            changes = changes + 1
        template = template+1
        if(template==5):
            template = 1
        print("iteration: ", iteration, "changes: ", changes)
        total_changes = total_changes + changes

    print("total_changes: ", total_changes)
    return thin_image

def hilditch(image):
    Image_Thinned=sparse.dok_matrix((image.shape[0]+2,image.shape[1]+2), dtype=bool)
    Image_Thinned[1:image.shape[0]+1,1:image.shape[1]+1]=image.copy()
    changing1 = 1
    i = 0
    while changing1:
        changes_occured = 0
        changing1 = []
        for pos in  Image_Thinned.keys():
            x=pos[0]
            y=pos[1]
            P2,P3,P4,P5,P6,P7,P8,P9 = n = neighbours(x, y, Image_Thinned)
            condition3 = (P4 * P6 * P8 == 0) or (zeroToOne(Image_Thinned,x-1,y-1)!=1)#p2
            condition4 = (P4 * P6 * P6 == 0) or (zeroToOne(Image_Thinned,x-1,y+1)!=1)#p4
            if (Image_Thinned[x,y] == 1 and 2 <= sum(n) <= 6 and transitions(n) == 1 and condition3 ==1 and condition4==1):
                    changing1.append((x,y))
        for x, y in changing1: 
            Image_Thinned[x,y] = 0
            changes_occured = changes_occured + 1

        i = i + 1
        print("Iteration: ", i , "changes_occured: ", changes_occured)
    return Image_Thinned


def neighbours(x,y,image):
    "Return 8-neighbours of image point P1(x,y), in a clockwise order"
    img = image
    x_1, y_1, x1, y1 = x-1, y-1, x+1, y+1
    return [ img[x_1,y], img[x_1,y1], img[x,y1], img[x1,y1],     # P2,P3,P4,P5
                img[x1,y], img[x1,y_1], img[x,y_1], img[x_1,y_1] ]    # P6,P7,P8,P9

def transitions(neighbours):
    "No. of 0,1 patterns (transitions from 0 to 1) in the ordered sequence"
    n = neighbours + neighbours[0:1]      # P2, P3, ... , P8, P9, P2
    return sum( (n1, n2) == (0, 1) for n1, n2 in zip(n, n[1:]) )  # (P2,P3), (P3,P4), ... , (P8,P9), (P9,P2)

def zhangSuen(image):
    def neighbours(x,y,image):
        "Return 8-neighbours of image point P1(x,y), in a clockwise order"
        img = image
        x_1, y_1, x1, y1 = x-1, y-1, x+1, y+1
        return [ img[x_1,y], img[x_1,y1], img[x,y1], img[x1,y1],     # P2,P3,P4,P5
                    img[x1,y], img[x1,y_1], img[x,y_1], img[x_1,y_1] ]    # P6,P7,P8,P9

    def transitions(neighbours):
        "No. of 0,1 patterns (transitions from 0 to 1) in the ordered sequence"
        n = neighbours + neighbours[0:1]      # P2, P3, ... , P8, P9, P2
        return sum( (n1, n2) == (0, 1) for n1, n2 in zip(n, n[1:]) )  # (P2,P3), (P3,P4), ... , (P8,P9), (P9,P2)
    "the Zhang-Suen Thinning Algorithm"
    Image_Thinned=sparse.dok_matrix((image.shape[0]+2,image.shape[1]+2), dtype=bool)
    for pos in  image.keys():
        Image_Thinned[pos[0]+1,pos[1]+1]=image[pos[0],pos[1]]
    changing1 = changing2 = 1        #  the points to be removed (set as 0)
    while changing1 or changing2:   #  iterates until no further changes occur in the image
        # Step 1
        changing1 = []
        for pos in  Image_Thinned.keys():
            x=pos[0]
            y=pos[1]
            P2,P3,P4,P5,P6,P7,P8,P9 = n = neighbours(x, y, Image_Thinned)
            if (Image_Thinned[x,y] == 1     and    # Condition 0: Point P1 in the object regions 
                2 <= sum(n) <= 6   and    # Condition 1: 2<= N(P1) <= 6
                transitions(n) == 1 and    # Condition 2: S(P1)=1  
                P2 * P4 * P6 == 0  and    # Condition 3   
                P4 * P6 * P8 == 0):         # Condition 4
                changing1.append((x,y))
        for x, y in changing1: 
            Image_Thinned[x,y] = 0
        # Step 2
        changing2 = []
        for pos in  Image_Thinned.keys():
            x=pos[0]
            y=pos[1]
            P2,P3,P4,P5,P6,P7,P8,P9 = n = neighbours(x, y, Image_Thinned)
            if (Image_Thinned[x,y] == 1   and        # Condition 0
                2 <= sum(n) <= 6  and       # Condition 1
                transitions(n) == 1 and      # Condition 2
                P2 * P4 * P8 == 0 and       # Condition 3
                P2 * P6 * P8 == 0):            # Condition 4
                changing2.append((x,y))    
        for x, y in changing2: 
            Image_Thinned[x,y] = 0
        print(len(changing1),len(changing2))
    return Image_Thinned

In [60]:
skeleton_doc_1_dilated=dilate(skeleton_doc_small)

In [62]:
cProfile.run('dilate(skeleton_doc_small)')

         28916650 function calls in 21.188 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
  1204852    0.728    0.000    5.154    0.000 <__array_function__ internals>:2(ndim)
   602424    0.197    0.000    0.876    0.000 <frozen importlib._bootstrap>:1009(_handle_fromlist)
   602424    0.362    0.000    0.533    0.000 <frozen importlib._bootstrap>:416(parent)
        1    1.198    1.198   21.167   21.167 <ipython-input-38-23a69b96267b>:1(dilate)
        1    0.021    0.021   21.188   21.188 <string>:1(<module>)
  1807276    0.519    0.000    2.630    0.000 _asarray.py:14(asarray)
   602424    1.518    0.000   16.445    0.000 _index.py:126(_validate_indices)
   602424    2.715    0.000    8.384    0.000 _index.py:245(_unpack_index)
   602424    0.774    0.000    0.832    0.000 _index.py:291(_check_ellipsis)
  1204848    1.086    0.000    1.757    0.000 _index.py:339(_first_element_bool)
  1204848    0.699    0.000    2.618 

In [64]:
import cProfile
cProfile.run('zhangSuen(skeleton_doc_1_dilated)')

94923 94503
12661 36
11 0
0 0
         392978448 function calls in 327.674 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
 17003766   10.259    0.000   72.536    0.000 <__array_function__ internals>:2(ndim)
  8501881    2.824    0.000   12.293    0.000 <frozen importlib._bootstrap>:1009(_handle_fromlist)
  8501881    4.971    0.000    7.443    0.000 <frozen importlib._bootstrap>:416(parent)
        1   11.807   11.807  327.657  327.657 <ipython-input-63-112c7a9696c7>:134(zhangSuen)
   862271    3.653    0.000  216.853    0.000 <ipython-input-63-112c7a9696c7>:135(neighbours)
   708445    1.685    0.000   22.571    0.000 <ipython-input-63-112c7a9696c7>:142(transitions)
  6376005   19.271    0.000   19.271    0.000 <ipython-input-63-112c7a9696c7>:145(<genexpr>)
        1    0.017    0.017  327.674  327.674 <string>:1(<module>)
 17475554    5.247    0.000   26.061    0.000 _asarray.py:14(asarray)
  8501881   21.449    0.000  2

In [39]:
skeleton_doc_1_thinned=zhangSuen(skeleton_doc_1_dilated)

1989028 1982302
253320 872
245 14
3 4
1 1
0 0


In [40]:
from extract_graph import from_sparse_to_graph, generate_nx_graph
graph_tab_1_reconnected=from_sparse_to_graph(skeleton_doc_1_thinned)
nx_graph_1,pos_1=generate_nx_graph(graph_tab_1_reconnected)

In [41]:
connected_small=list(nx.connected_components(nx_graph_1))

In [51]:
len_connected = [len(connected) for connected in connected_small]

In [48]:
nx_graph_clean=prune_graph(nx_graph_1)

In [57]:
from extract_graph import generate_skeleton
skel_clean=generate_skeleton(nx_graph_clean,dim=skeleton_doc_1_thinned.shape)

In [58]:
from extract_graph import from_sparse_to_graph
panda_graph = from_sparse_to_graph(skel_clean)

In [56]:
skeleton_doc_1_thinned.shape

(20687, 45556)

In [54]:
pos_1[2854]

array([12603, 11744])

In [52]:
len_connected

[6673, 4, 4, 4, 4, 118, 6, 8, 4, 10, 4, 4, 6]

In [46]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax.hist(len_connected)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [33]:
plot_nodes_from_list(list(connected_small[11]),pos_1,skeleton_doc_1_thinned.todense())

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [38]:
connected_small=list(nx.connected_components(nx_graph_small.todense()))

In [39]:
len(connected_small)

642

In [40]:
skeleton_doc_small

<5000x5000 sparse matrix of type '<class 'numpy.float64'>'
	with 66952 stored elements in Dictionary Of Keys format>

In [43]:
list(connected_small[1])

[160, 123]

In [35]:
pos_small[1306]

array([2240,  413])

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax.imshow(skeleton_doc_small.todense())
plt.show()

In [11]:
graph_tab=from_sparse_to_graph(skeleton_doc)

In [ ]:
import cProfile
cProfile.run('from_sparse_to_graph(skeleton_doc)')

In [23]:
nx_graph,pos=generate_nx_graph(graph_tab)

In [13]:
nx_graph=prune_graph(nx_graph)

In [24]:
connected=list(nx.connected_components(nx_graph))

In [25]:
len(connected)

14100

In [ ]:
for edge in nx_graph.subgraph(connected[329]).copy().edges.data('pixel_list'):
    print(edge)

In [ ]:
def zsAlgoIterationOne(image):
    Image_Thinned=sparse.dok_matrix((image.shape[0]+2,image.shape[1]+2), dtype=bool)
    Image_Thinned[1:image.shape[0]+1,1:image.shape[1]+1]=image.copy()
    changing1 = changing2 = 1
    i = 0
    while changing1 or changing2:
        changes_occured = 0
        changing1 = []
        for pos in Image_Thinned.keys():
            x=pos[0]
            y=pos[1]
            P2,P3,P4,P5,P6,P7,P8,P9 = n = neighbours(x, y, Image_Thinned)
            if (Image_Thinned[x,y] == 1 and 2 <= sum(n) <= 6 and transitions(n) == 1 and P2 * P4 * P6 == 0  and P4 * P6 * P8 == 0):
                changing1.append((x,y))
        for x, y in changing1: 
            Image_Thinned[x,y] = 0
            changes_occured = changes_occured + 1
        changing2 = []
        for pos in Image_Thinned.keys():
            x=pos[0]
            y=pos[1]
            P2,P3,P4,P5,P6,P7,P8,P9 = n = neighbours(x, y, Image_Thinned)
            if (Image_Thinned[x,y] == 1 and 2 <= sum(n) <= 6 and transitions(n) == 1 and P2 * P4 * P8 == 0  and P2 * P6 * P8 == 0):
                changing2.append((x,y))    
        for x, y in changing2: 
            Image_Thinned[x,y] = 0
            changes_occured = changes_occured + 1
        i = i + 1
        print("Iteration: ", i , "changes_occured: ", changes_occured)
    return Image_Thinned

def neighbours(x,y,image):
    img = image
    x_1, y_1, x1, y1 = x-1, y-1, x+1, y+1
    return [ img[x_1,y], img[x_1,y1], img[x,y1], img[x1,y1], img[x1,y], img[x,y_1], img[x,y_1], img[x_1,y_1] ]

def transitions(neighbours):
    n = neighbours + neighbours[0:1]
    return sum( (n1, n2) == (0, 1) for n1, n2 in zip(n, n[1:]) )

def foregroundPixels(image):
    fgp = 0
    row, col = image.shape
    for pos in image.keys():
        i=pos[0]
        j=pos[1]
        if(image[i][j]==1):
            fgp = fgp + 1
    return fgp

from copy import copy

In [146]:
time=cProfile.run('generate_nx_graph(graph_tab)')

         14812846 function calls (14672662 primitive calls) in 235.492 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
   140184    0.097    0.000    0.274    0.000 <frozen importlib._bootstrap>:1009(_handle_fromlist)
        1    0.025    0.025  235.492  235.492 <string>:1(<module>)
        2    0.000    0.000    0.000    0.000 _asarray.py:14(asarray)
    93456    0.042    0.000    0.042    0.000 _dtype.py:24(_kind_name)
    93456    0.039    0.000    0.039    0.000 _dtype.py:307(_name_includes_bit_suffix)
    93456    0.231    0.000    0.332    0.000 _dtype.py:321(_name_get)
   140184    0.054    0.000    0.192    0.000 abc.py:137(__instancecheck__)
        2    0.000    0.000    0.000    0.000 base.py:1032(__iter__)
   186912    0.184    0.000    0.184    0.000 base.py:1751(is_floating)
   186912    0.127    0.000    0.561    0.000 base.py:256(is_dtype)
   186912    0.272    0.000    1.142    0.000 base.py:2847(get_loc)
